In [4]:
from operator import itemgetter

def format_docs(docs):
    return "\n\n".join(doc for doc in docs)

a = {"question": "sdf", "documents": ["bgf", "fgsf", "fsfs"]}
tmp = itemgetter("question")
tmp1 = lambda input: format_docs(input["documents"])

In [5]:
tmp1(a)

'bgf\n\nfgsf\n\nfsfs'

In [2]:
tmp(a)

'sdf'